In [1]:
from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [2]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results = 2, doc_content_chars_max = 500)
arxiv = ArxivQueryRun(api_wrapper = api_wrapper_arxiv)
arxiv.name

'arxiv'

In [3]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results = 2, doc_content_chars_max = 500)
wiki = WikipediaQueryRun(api_wrapper = api_wrapper_wiki)
wiki.name

'wikipedia'

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ReAct Agent"

In [11]:
def add(a:int,b:int) -> int:
    '''
    Add the two integers a and b
    where a is the 1st integer
    and b is the 2nd integer
    '''
    return a + b

def multiply(a:int,b:int) -> int:
    '''
    Multiply the two integers a and b
    where a is the 1st integer
    and b is the 2nd integer
    '''
    return a * b

def div(a:int,b:int) -> float:
    '''
    Divide the two integers a and b
    where a is the 1st integer
    and b is the 2nd integer
    '''
    return a / b

In [12]:
tools = [arxiv, wiki, add, multiply, div]

In [13]:
from langchain_groq import ChatGroq

llm = ChatGroq(model = "gemma2-9b-it", temperature = 0)

llm_with_tools = llm.bind_tools(tools)

In [15]:
from typing_extensions import TypedDict
from typing import Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


In [17]:
def tool_calling_llm(state:State):
    return{"messages":[llm_with_tools.invoke(state["messages"])]}

In [20]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

builder = StateGraph(State)

builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm", tools_condition)
builder.add_edge("tools", "tool_calling_llm")

graph = builder.compile()

In [27]:
from langchain_core.messages import HumanMessage

messages = graph.invoke({"messages":HumanMessage(content = "What is attention?, then add 5 plus 5 and then multiply by 3 and then 10")})

for m in messages["messages"]:
    m.pretty_print()


================================ Human Message =================================

What is attention?, then add 5 plus 5 and then multiply by 3 and then 10
================================== Ai Message ==================================
Tool Calls:
  wikipedia (b9s348scf)
 Call ID: b9s348scf
  Args:
    query: attention
================================= Tool Message =================================
Name: wikipedia

Page: Attention
Summary: Attention or focus, is the concentration of awareness on some phenomenon to the exclusion of other stimuli. It is the selective concentration on discrete information, either subjectively or objectively. William James (1890) wrote that "Attention is the taking possession by the mind, in clear and vivid form, of one out of what seem several simultaneously possible objects or trains of thought. Focalization, concentration, of consciousness are of its essence." Attention has
================================== Ai Message ==================================